In [1]:
import requests
import bs4
from bs4 import BeautifulSoup as bs
import urllib.request as ureq
import csv

In [3]:
#defining the url and using the ureq to read the website in html. This can be checked by print(canadavisa)

In [2]:
Canadavisa_url = 'https://www.canadavisa.com/canada-immigration-discussion-board/'
canadavisa = ureq.urlopen(Canadavisa_url).read()


In [4]:
#all topics are in tag 'h3' - using bs to find all data in h3

In [3]:
soup = bs(canadavisa, 'html.parser')

topics = soup.find_all('h3', {'class':'nodeTitle'})
titles = list(map(lambda h: h.text.strip(), topics))
print(topics)
print(titles)

titles_and_links = dict(map(lambda h: (h.text.strip(), h.find('a') ['href']), topics))


[<h3 class="nodeTitle"><a data-description="#nodeDescription-9" href=".#immigration-to-canada.9">Immigration to Canada</a></h3>, <h3 class="nodeTitle"><a data-description="" href="forums/express-entry-expression-of-interest.54/">Express Entry / Expression of Interest</a>
</h3>, <h3 class="nodeTitle"><a data-description="" href="forums/skilled-worker-professional-immigration.4/">Skilled Worker / Professional Immigration</a>
</h3>, <h3 class="nodeTitle"><a data-description="" href="forums/canadian-experience-class.52/">Canadian Experience Class</a>
</h3>, <h3 class="nodeTitle"><a data-description="" href="forums/provincial-nomination-program-immigration.6/">Provincial Nomination Program Immigration</a>
</h3>, <h3 class="nodeTitle"><a data-description="" href="forums/family-class-sponsorship.5/">Family Class Sponsorship</a>
</h3>, <h3 class="nodeTitle"><a data-description="" href="forums/caregiver-pathways.65/">Caregiver Pathways</a>
</h3>, <h3 class="nodeTitle"><a data-description="" hre

In [6]:
#lambda function to create a list of all text only titles from 'topics'

In [7]:

# print(titles)

In [8]:
#the titles of topics and their associated links

In [9]:
# print(titles_and_links)

In [10]:
#creating a csv file to open the data in an excel sheet

In [4]:
with open('titles_and_links.csv', mode='w') as csv_file:
    fieldnames = ['title', 'link']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

    writer.writeheader()
    for key, value in titles_and_links.items():
        writer.writerow({'title': key, 'link': value})


In [5]:
message_contents = list()
date_contents = list()
username_contents = list()
topic_contents = list()
subtopic_contents = list()
counter = 0
for key, value in titles_and_links.items():
    counter = counter + 1
    if counter == 1:
        print('inside loop',key,value)
        continue

    # For debugging only - COMMENT OUT LATER
    if counter > 2:
        print('inside loop',key,value)
        continue

    topic_url = Canadavisa_url + value 
    #print(topic_url)
    topic_title = key
    try:
        topic_content = bs(ureq.urlopen(topic_url).read(), 'html.parser')
        subtopics = topic_content.find_all('h3', {'class':'title'})
        subtopic_titles_and_links = dict(map(lambda subtopic: (subtopic.text.strip(), subtopic.find('a') ['href']), subtopics))
        subtopic_counter = 0
        for st_key, st_value in subtopic_titles_and_links.items():
            subtopic_counter = subtopic_counter + 1
            if subtopic_counter > 1:
                print('inside loop',key,value)
                continue
            subtopic_url = Canadavisa_url + st_value
            subtopic_title = st_key
            print(topic_title, subtopic_title)
            subtopic_content = bs(ureq.urlopen(subtopic_url).read(), 'html.parser')
            pages = subtopic_content.find_all('nav')
            for page in pages:
                link = page.find('a', {'class': 'PageNavNext'})
                print(link)
    except HTTPError:
        print("Can't find this page... Moving on")



inside loop Immigration to Canada .#immigration-to-canada.9
Express Entry / Expression of Interest Express Entry Step by step instructions.
None
None
None
<a class="PageNavNext ">→</a>
<a class="PageNavNext ">→</a>
None
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-interest.54/
inside loop Express Entry / Expression of Interest forums/express-entry-expression-of-i

In [6]:
message_contents = list()
date_contents = list()
username_contents = list()
topic_contents = list()
subtopic_contents = list()
counter = 0
print(titles_and_links)
for key, value in titles_and_links.items():
    counter = counter + 1
    if counter == 1:
        print('inside loop',key,value)
        continue

    # For debugging only - COMMENT OUT LATER
    if counter > 2:
        print('inside loop',key,value)
        continue

    topic_url = Canadavisa_url + value 
    print(topic_url)
    topic_title = key
    try:
        topic_content = bs(ureq.urlopen(topic_url).read(), 'html.parser')
        subtopics = topic_content.find_all('h3', {'class':'title'})
        subtopic_titles_and_links = dict(map(lambda subtopic: (subtopic.text.strip(), subtopic.find('a') ['href']), subtopics))
        for st_key, st_value in subtopic_titles_and_links.items():
            subtopic_url = Canadavisa_url + st_value
            subtopic_title = st_key
            print(topic_title, subtopic_title)
            subtopic_content = bs(ureq.urlopen(subtopic_url).read(), 'html.parser')
            messages = subtopic_content.find_all('div', {'class':'messageContent'})
            message_dates = subtopic_content.find_all('span', {'class':'DateTime'})
            message_usernames = subtopic_content.find_all('a', {'class':'username'})
            message_list = list(map(lambda message: (message.find('blockquote').decode_contents().strip()), messages))
            usernames_list = list(map(lambda a: a.text.strip(), message_usernames))
            for idx, m in enumerate(message_list):
                message_date = ""
                try:
                    message_date = message_dates[idx]['title']
                except IndexError:
                    message_date = ""
                date_contents.append(message_date)
                message_username = ""
                try:
                    message_username = usernames_list[idx]
                except IndexError:
                    message_username = ""
                username_contents.append(message_username)
                message_contents.append(m)
                topic_contents.append(topic_title)
                subtopic_contents.append(subtopic_title)
    except HTTPError:
        print("Can't find this page... Moving on")


{'Immigration to Canada': '.#immigration-to-canada.9', 'Express Entry / Expression of Interest': 'forums/express-entry-expression-of-interest.54/', 'Skilled Worker / Professional Immigration': 'forums/skilled-worker-professional-immigration.4/', 'Canadian Experience Class': 'forums/canadian-experience-class.52/', 'Provincial Nomination Program Immigration': 'forums/provincial-nomination-program-immigration.6/', 'Family Class Sponsorship': 'forums/family-class-sponsorship.5/', 'Caregiver Pathways': 'forums/caregiver-pathways.65/', 'Quebec Immigration': 'forums/quebec-immigration.55/', 'Business Immigration': 'forums/business-immigration.7/', 'IELTS - CELPIP - TEF - TCF - Language Testing': 'forums/ielts-celpip-tef-tcf-language-testing.53/', 'Refugees and Asylum': 'forums/refugees-and-asylum.56/', 'General - All Canadian Immigration': 'forums/general-all-canadian-immigration.13/', 'Moving to Canada from the U.S.': '.#moving-to-canada-from-the-u-s.31', 'Permanent Residence in Canada': 'fo

Express Entry / Expression of Interest 办理高贵林学院大学文凭毕业证书Q/微457202606CC毕业证成绩单学历认证1:1制作电子图修改G  PA成绩CoQuitlam College
Express Entry / Expression of Interest 办理百年理工学院大学文凭毕业证书Q/微457202606SAIT毕业证成绩单学历认证1:1制作电子图  修改GPA成绩Centennial College
Express Entry / Expression of Interest 办理兰加拉学院大学文凭毕业证书Q/微457202606Langara毕业证成绩单学历认证1:1制作电  子图修改GPA成绩Langara College
Express Entry / Expression of Interest June 2018 AOR - join here
Express Entry / Expression of Interest Did anyone receive requests for PPR from Los Angeles Office
Express Entry / Expression of Interest 办理亚岗昆学院大学文凭毕业证书Q/微457202606NGC毕业证成绩单学历认证1:1制作电子图修改  GPA成绩AlgonQuin
Express Entry / Expression of Interest 办理北英属哥伦比亚大学文凭毕业证书Q/微457202606UNBC毕业证成绩单学历认证1:1制作电子  图修改GPA成绩University of Northern British Columbia
Express Entry / Expression of Interest 办理不列颠哥伦比亚理工学院大学文凭毕业证书Q/微457202606BCIT毕业证成绩单学历认证1:1  制作电子图修改GPA成绩British Columbia Institute of Technolo
Express Entry / Expression of Interest 办理尼亚加拉学院大学文凭毕业证书Q/微457202606NC毕业证成绩单学历认证1:1制作电子图修改  GPA成绩Niagara

In [ ]:

with open('messages.csv', mode='w') as csv_file:
    fieldnames = ['topic', 'subtopic', 'date', 'username', 'message']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()
    for idx, value in enumerate(message_contents):
        date_label = date_contents[idx]
        username_label = username_contents[idx]
        topic_title = topic_contents[idx]
        subtopic_title = subtopic_contents[idx]
        writer.writerow({'topic': topic_title, 'subtopic': subtopic_title, 'date': date_label, 'username': username_label, 'message': value})


In [22]:
import pandas

In [8]:
print(topic_title, subtopic_title, messages

Express Entry / Expression of Interest Thank you  >>>>>>>>>  Legalfalcon  <<<<<<<<<<< [<div class="messageContent">
<article>
<blockquote class="messageText SelectQuoteContainer ugc baseHtml">
<b>@legalfalcon</b><br/>
<br/>
<br/>
It is really Generous of you stay back on Forum to help others....<br/>
<br/>
I have a very common question for which i would love to get response from you. <br/>
My background status changed from Not Started to Not needed at this time, can you please help me understand what could be the possible reasons for that, and when does it happens
						<div class="messageTextEndMarker"> </div>
</blockquote>
</article>
</div>, <div class="messageContent">
<article>
<blockquote class="messageText SelectQuoteContainer ugc baseHtml">
<div class="bbCodeBlock bbCodeQuote" data-author="Honeyaustin">
<aside>
<div class="attribution type">Honeyaustin said:
				
			</div>
<blockquote class="quoteContainer"><div class="quote"><b>@legalfalcon</b><br/>
<br/>
<br/>
It is really Gen